# Load Data

In [1]:
import os
os.chdir("../../scVI/")
os.getcwd()

import pickle

from scvi.dataset.dataset import GeneExpressionDataset
from scvi.dataset.dataset10X import Dataset10X
import numpy as np
import pandas as pd

In [2]:
CSF = Dataset10X('CSF_data/',save_path='/data/yosef2/users/chenling/CSF',remote=False,genecol=0)

celltypes = pd.read_csv('/data/yosef2/users/chenling/CSF/CSF_data/celltypes.txt')
donor = np.asarray(celltypes['NA'])
celltype = np.asarray(celltypes['clustering'])
subtype = np.asarray(celltypes['fineclusters'])

CSF.cell_types,CSF.labels = np.unique(celltype,return_inverse=True)
CSF.labels = CSF.labels.reshape(len(CSF),1)
CSF.batch_names,CSF.batch_indices = np.unique(donor,return_inverse=True)
CSF.batch_indices = CSF.batch_indices.reshape(len(CSF.batch_indices),1)
CSF.n_batches = len(np.unique(CSF.batch_indices))
CSF.n_labels = len(np.unique(CSF.labels))

CSF.batch_names

patient_id = ['MS19270', 'MS49131','MS71658','MS60249','MS74594', 'PTC32190','PTC41540','PTC85037','PST83775','PST95809']
dataset = []
celltype='PBMCs'
for id in patient_id:
    data = Dataset10X('{0}/{1}/GRCh38'.format(id, celltype),
                save_path='/data/yosef2/users/chenling/CSF/CSF_data/Gerd/',remote=False,genecol=1)
    dataset.append(data)

PBMC = GeneExpressionDataset.concat_datasets(*dataset)


PBMC.gene_names

gene_dataset = GeneExpressionDataset.concat_datasets(CSF,PBMC)

Preprocessing dataset
Finished preprocessing dataset
Preprocessing dataset
Finished preprocessing dataset
Preprocessing dataset
Finished preprocessing dataset
Preprocessing dataset
Finished preprocessing dataset
Preprocessing dataset
Finished preprocessing dataset
Preprocessing dataset
Finished preprocessing dataset
Preprocessing dataset
Finished preprocessing dataset
Preprocessing dataset
Finished preprocessing dataset
Preprocessing dataset
Finished preprocessing dataset
Preprocessing dataset
Finished preprocessing dataset
Preprocessing dataset
Finished preprocessing dataset
Keeping 33694 genes
Keeping 10266 genes


# Save Data

In [4]:
with open('../CSF/Notebooks/dataset/all_datasets.pkl', 'wb') as f:
    pickle.dump(gene_dataset, f)


# Save data object of just CD4 data

In [7]:
celllables = np.load('../CSF/clustering/celllabels.npy')
isCD4 = (celllables =='CD4')
gene_dataset.update_cells(isCD4)

Downsampling from 65326 to 25105 cells


In [8]:
with open('../CSF/Notebooks/dataset/CD4only.pkl', 'wb') as f:
    pickle.dump(gene_dataset, f)


# write to matrix for edgeR

In [1]:
import os
os.chdir("../../scVI/")
os.getcwd()

import pickle

In [2]:
with open('../CSF/Notebooks/dataset/all_datasets.pkl', 'rb') as f:
    gene_dataset = pickle.load(f)


In [3]:
gene_dataset.X.shape

(65326, 10266)

In [5]:
import numpy as np

In [6]:
isCSF = np.load('../CSF/Notebooks/meta/isCSF.npy')

In [9]:
np.sum(isCSF)

22357

In [10]:
np.sum(isCSF==False)

42969

In [3]:
from scipy.io import mmwrite
mmwrite('../CSF/RFiles/all_data.mtx',gene_dataset.X)

In [5]:
from numpy import savetxt
savetxt('../CSF/RFiles/genenames.csv', gene_dataset.gene_names,delimiter=',', fmt='%s')

In [6]:
savetxt('../CSF/RFiles/batchid.csv', gene_dataset.batch_indices.ravel(),delimiter=',', fmt='%i')

In [11]:
import numpy as np
tissue=np.concatenate([np.repeat('CSF',8),np.repeat('PBMC',10)])
states=np.asarray(['MS','MS','MS','MS','control','control','control','control',
                   'MS','MS','MS','MS','MS','control','control','control','control','control'])


batchid = gene_dataset.batch_indices.ravel().astype('int')
isMS = np.asarray([states[i]=='MS' for i in batchid])
isCSF = np.asarray([tissue[i]=='CSF' for i in batchid])


In [15]:
_,ncells= np.unique(batchid,return_counts=True)
[np.mean(ncells[(tissue=='CSF') & (states=='MS')]),np.std(ncells[(tissue=='CSF') & (states=='MS')])]

[2413.0, 1038.069602676044]

In [17]:
[np.mean(ncells[(tissue=='CSF') & (states=='control')]),np.std(ncells[(tissue=='CSF') & (states=='control')])]

[3176.25, 2811.307469399247]

In [10]:
savetxt('../CSF/RFiles/isMS.csv', isMS, delimiter=',', fmt='%s')
savetxt('../CSF/RFiles/isCSF.csv', isCSF, delimiter=',', fmt='%s')

# write to data for vision

In [1]:
import os
os.chdir("../../scVI/")
os.getcwd()

import pickle

In [2]:
with open('../CSF/Notebooks/dataset/CD4only.pkl', 'rb') as f:
    gene_dataset = pickle.load(f)


In [3]:
from scipy.io import mmwrite
mmwrite('../CSF/RFiles/CD4data.mtx',gene_dataset.X)